In [1]:
# default_exp Program

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# School Model

> API details.

In [3]:
#export
from collections import defaultdict

In [4]:
#export

class Record:
    
    def __init__(self, name,key = 0,sort = 0):
        self.name = name
        self.key = key
        self.sort = sort
        
    def adjustEmptyKey(self,others):
        if self.key < 1:
            maxVal = max([x.key for x in others])
            self.key = maxVal + 1
        if self.sort == 0:
            self.sort = max([x.sort for x in others]) + 1

    def adjustKey(self,x):
        props = x.split('|')
        if len(props) > 2:
            self.name = props[0]
            self.key = int(props[1])
            self.sort = int(props[2])
            
    def keyString(self):
        return self.name + "|" + str(self.key) + "|" + str(self.sort)
    
    def toDict(self):
        return {"key" : self.keyString() }
    
    def fromDict(s):
        ret = Record("")
        ret.adjustKey(s["key"])
        return ret
    
    def __repr__(self):
        return self.keyString()
    
    def printKeys(x):
        s = sorted(x, key=lambda l: l.sort)
        return [y.keyString() for y in s]
    
    def appendUnique(x,others):
        if x.key > 0:
            others = [y for y in others if x.key != y.key]
        others.append(x)
        x.adjustEmptyKey(others)
        return others
    

In [5]:
#hide
recs = [Record(x) for x in ["one","two","three"]]
Record.printKeys(recs),recs[0]

(['one|0|0', 'two|0|0', 'three|0|0'], one|0|0)

In [6]:
#hide
for x in recs:
    x.adjustEmptyKey(recs)
Record.printKeys(recs)

['one|1|1', 'two|2|2', 'three|3|3']

In [7]:
#hide
x = recs[0]
x.adjustKey('higher|10|1')
recs.append(Record("newGuy"))
for x in recs:
    x.adjustEmptyKey(recs)
Record.printKeys(recs)

['higher|10|1', 'two|2|2', 'three|3|3', 'newGuy|11|4']

In [8]:
#hide
addMe = []
for x in ['another2|2|2', 'another|3|3', 'oldguy|11|4',"someoneNew","someoneBlue"]:
    rec = Record(x)
    rec.adjustKey(x)
    addMe.append(rec)

Record.printKeys(addMe),Record.printKeys( recs)

(['someoneNew|0|0',
  'someoneBlue|0|0',
  'another2|2|2',
  'another|3|3',
  'oldguy|11|4'],
 ['higher|10|1', 'two|2|2', 'three|3|3', 'newGuy|11|4'])

In [9]:
for x in addMe:
     recs = Record.appendUnique(x,recs)
x.keyString(), Record.printKeys(recs)

('someoneBlue|13|6',
 ['higher|10|1',
  'another2|2|2',
  'another|3|3',
  'oldguy|11|4',
  'someoneNew|12|5',
  'someoneBlue|13|6'])

In [10]:
aFoo = recs[0]
aBarr= Record.fromDict(aFoo.toDict())
aBarr

higher|10|1

In [11]:
#export
class Lesson(Record):
    def __init__(self, name,  filesName, key = 0,sort = 0):
        super().__init__(name,key,sort)
        self.fileName = filesName
        self.modifyTime = float(0)
        self.indexPath = []
   
    def sBase(self):
        if self.modifyTime > 0:
            return  self.modifyTime
        return float(self.sort)
    
    def keyString(self):
        return  super().keyString() + "|" + self.fileName  
    
    def adjustKey(self,x):
        super().adjustKey(x)
        props = x.split('|')
        if len(props) > 3:
            self.fileName = props[3]
        
    def appendUniqueFile(x,others):
        ret = [y for y in others if x.fileName != y.fileName]
        if len(ret) != len(others):
            return others
        return Record.appendUnique(x,others)
    
    def toDict(self):
        ret = super().toDict()
        ret["modifyTime"] = self.modifyTime
        ret["fileName"] = self.fileName
        ret["indexPath"] = self.indexPath
        return ret
    
    def fromDict(s):
        ret = Lesson("","")
        ret.adjustKey(s["key"])
        ret.modifyTime = s["modifyTime"]
        ret.fileName = s["fileName"]
        ret.indexPath = s["indexPath"]
        return ret
    
    def __str__(self):
        return "[" + str(self.key) + "]" + self.fileName
    

In [12]:
#hide
less = [Lesson(x,x +".mp4") for x in ["one","two","three"]]
Record.printKeys(less)

['one|0|0|one.mp4', 'two|0|0|two.mp4', 'three|0|0|three.mp4']

In [13]:
#hide
less = Lesson.appendUniqueFile(Lesson("two","two.pdf"),less)
less = Lesson.appendUniqueFile(Lesson("one","one.mp4"),less)
aLess = Lesson("", "")
aLess.adjustKey('three|1|1|three.mp4')
less = Lesson.appendUniqueFile(aLess,less)
Record.printKeys(less)

['one|0|0|one.mp4',
 'two|0|0|two.mp4',
 'three|0|0|three.mp4',
 'two|1|1|two.pdf']

In [14]:
#hide
s = aLess.toDict()
s
cLess = Lesson.fromDict(s)
Record.printKeys([cLess]),s

(['three|1|1|three.mp4'],
 {'key': 'three|1|1|three.mp4',
  'modifyTime': 0.0,
  'fileName': 'three.mp4',
  'indexPath': []})

In [15]:
#export

class Subject(Record):
    def __init__(self, name,key = 0,sort = 0):
        super().__init__(name,key,sort) 
        self.lessons = []
        self.sequences = defaultdict(list)
        self.lookup = {}
        self.img = ""
    
    def addLesson(self,x):
        self.lessons = Lesson.appendUniqueFile(x,self.lessons)
        self.lookup[x.key] = x
     
    def toDict(self):
        ret = super().toDict()
        ret["lessons"] = [x.toDict() for x in self.lessons]
        seq = {}
        for key, value in self.sequences.items():
            seq[key] = value
        ret["sequences"] = seq
        ret["img"] = self.img 
        return ret
    
    def fromDict(d):
        ret = Subject("","")
        ret.adjustKey(d["key"])
        ret.lessons = [ Lesson.fromDict(s) for s in d["lessons"]]
        
        for x in ret.lessons:
            ret.lookup[x.key] = x
            
        for key, value in d["sequences"].items():
            ret.sequences[key] = value
        ret.img = d["img"]    
        return ret
    
    def __repr__(self):
  
        return self.name + "; [" + ",".join( Record.printKeys(self.lessons)) + "]"
    
    def __str__(self):
        return '__str__ for Car'

In [16]:
#hide
aless = Lesson("Intro to Network Flow","flow.mp4")
bless = Lesson("Imnune","sick.mp4")
grad = Subject("operations research")
grad.addLesson(aless)
grad.addLesson(bless)
grad.sequences["main"] = [1,2]
bgrad = Subject.fromDict(grad.toDict())
grad.toDict(),bgrad.toDict(),bgrad.lookup


({'key': 'operations research|0|0',
  'lessons': [{'key': 'Intro to Network Flow|1|1|flow.mp4',
    'modifyTime': 0.0,
    'fileName': 'flow.mp4',
    'indexPath': []},
   {'key': 'Imnune|2|2|sick.mp4',
    'modifyTime': 0.0,
    'fileName': 'sick.mp4',
    'indexPath': []}],
  'sequences': {'main': [1, 2]},
  'img': ''},
 {'key': 'operations research|0|0',
  'lessons': [{'key': 'Intro to Network Flow|1|1|flow.mp4',
    'modifyTime': 0.0,
    'fileName': 'flow.mp4',
    'indexPath': []},
   {'key': 'Imnune|2|2|sick.mp4',
    'modifyTime': 0.0,
    'fileName': 'sick.mp4',
    'indexPath': []}],
  'sequences': {'main': [1, 2]},
  'img': ''},
 {1: Intro to Network Flow|1|1|flow.mp4, 2: Imnune|2|2|sick.mp4})

In [17]:
#export
class Program(Record):
    def __init__(self, name, path = ".",key = 0,sort = 0 ):
        super().__init__(name,key,sort)
        self.subjects = []
        self.title = name
        self.lookup = {}
        self.img = ""
    
    def toDict(self):
        ret = super().toDict()
        ret["subjects"] = [x.toDict() for x in self.subjects]
        ret["title"] = self.title
        ret["img"] = self.img
        return ret
    
    def fromDict(d):
        ret = Program("")
        ret.adjustKey(d["key"])
        ret.subjects = [Subject.fromDict(s) for s in d["subjects"]]
        ret.title = d["title"]
        for x in ret.subjects:
            ret.lookup[x.key] = x
        ret.img = d["img"]
        return ret
    
        
    def fixPaths(self):
        for progIndex, sub in self.lookup.items():
            for subIndex, lesson in sub.lookup.items():
                lesson.indexPath = [progIndex,subIndex]
    
    def addSubject(self,x):
        self.subjects = Subject.appendUnique(x,self.subjects)
        self.lookup[x.key] = x
    
    def __repr__(self):
        return self.name + "Program, Subjects[" + ";".join( Record.printKeys(self.subjects)) +"]"
    
    def __str__(self):
        return '__str__ for Car'

In [18]:
#hide
cmu = Program("CMU")
opre = Subject("operations research")
cmu.addSubject(opre)
for x in [aless,bless]:
    opre.addLesson(x)

less = []
                   
s =  Subject("networking")
for x in ["beer","pong","sand bunny"]:
    s.addLesson(Lesson(x,x + ".pdf"))
    s.addLesson(Lesson(x,x + ".mp4"))

cmu.addSubject(s)
Dartmouth = Program.fromDict(cmu.toDict())
Dartmouth.name = "Dartmouth"
Dartmouth,cmu

(DartmouthProgram, Subjects[operations research|1|1;networking|2|2],
 CMUProgram, Subjects[operations research|1|1;networking|2|2])

In [19]:
#hide
cmu.fixPaths()
#cmu.toDict(),cmu.lookup
for progIndex, sub in cmu.lookup.items():
    print(progIndex,sub.name,sub.key)

1 operations research 1
2 networking 2


In [21]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_StudySystem.ipynb.
Converted 02_DataMunge.ipynb.
Converted 03_Flask.ipynb.
Converted 05_staticGenerator.ipynb.
Converted index.ipynb.
